# **Proyecto Final - Data Science 2**
## **Prediccion de Pais Destino en AirBnB**

En esta entrega se cubren las siguentes rubricas:

1. Generar preguntas o hipótesis de interés sobre el dataset elegido para el proyecto final.
2. Crear gráficos que usen al menos tres variables y hacer un diagnóstico de lo que los mismos nos están diciendo.
3. Vincular los gráficos y análisis numéricos hechos hasta el momento con las preguntas de interés.
4. Identificar valores perdidos.

El dataset esta compuesto de una carpeta de archivos en formato CSV conteniendo informacion de usuarios de USA que realizan reservaciones en AirBnB para diferentes paises. Dicho Dataset puede descargarse de kaggle (solo despues de un muy engorroso proceso de alta de usuario) en la siguiente liga:

__https://www.kaggle.com/competitions/airbnb-recruiting-new-user-bookings/data__

El sitio web describe el Dataset con el siguiente parrafo:

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Dataset Description'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

In this challenge, you are given a list of users along with their demographics, web session records, and some summary statistics. You are asked to predict which country a new user's first booking destination will be. All the users in this dataset are from the USA.

There are 12 possible outcomes of the destination country: 'US', 'FR', 'CA', 'GB', 'ES', 'IT', 'PT', 'NL','DE', 'AU', 'NDF' (no destination found), and 'other'. Please note that 'NDF' is different from 'other' because 'other' means there was a booking, but is to a country not included in the list, while 'NDF' means there wasn't a booking.

The training and test sets are split by dates. In the test set, you will predict all the new users with first activities after 7/1/2014 (note: this is updated on 12/5/15 when the competition restarted). In the sessions dataset, the data only dates back to 1/1/2014, while the users dataset dates back to 2010. ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

## Pt. I Analisis de Metadatos

El dataset consta de 5 archivos en formato CSV que contienen unformacion de los usuarios y reservaciones en la plataforma AirBnb. Los archivos son los siguientes:

1. **age_gender_bkts**: contiene info. de la edad, pais, genero, ID y anyo de registro de los usuarios
2. **countries**: describe los paises destino de los diferentes usuarios, describe la ubicacion con latitud y longitud, distancia en km, descripcion del destino en km2, lenguaje del pais destino y distancia levenshtein del lenguaje
3. **sessions**: contiene informacion de sesion del usuario, userID, accion ejecutada, tipo de accion, detalle de accion, tipo de dispositivo y segundos transcurridos
4. **test_users**: dataset de 15 columnas con informacion de los regtistros como user ID, fecha de creacion del usuario, timestamp de actividad, fecha de la primera reserva, genero, edad, metodo de registro, flujo de registro, idioma, canal afiliado, proveedor afiliado, primer afiliado reconocido, tipo de aplicacion de registro y primer browser. Se usa para probar el modelo predictivo.
5. **train_users_2**: dataset de 16 columnas con informacion de los regtistros como user ID, fecha de creacion del usuario, timestamp de actividad, fecha de la primera reserva, genero, edad, metodo de registro, flujo de registro, idioma, canal afiliado, proveedor afiliado, primer afiliado reconocido, tipo de aplicacion de registro, primer browser y pais de destino. 

El objetivo de todo el dataset es que el programador se dedique a disenyar y entrenar un modelo predictivo para lograr saber cual es el siguiente pais que visitara un usuario registrado.

Los dos archivos de Excel relevantes para el estudio (a mi consideracion) son:

1. **train_set**
2. **test_set**


### Librerias Necesarias
Este proyecto empieza importando las librerias necesarias. Necesito bastantes librerias para llenar, organizar, procesar, y revisar estructuras de datos o Dataframes.

Procesamiento de Dataframes:

    1. Pandas
    2. NumPy
    3. datetime  

Generacion de Graficas:

    4. Matplotlib

Modelado de Machine Learning:

    5. sklearn LabelEncoder
    6. sklearn train_test_slpit
    7. sklearn RandomForestClassifier
    8. sklearn cross_val_score
    9. sklearn classification_report
    10. sklearn confusion_matrix,mean_absolute_error, mean_squared_error, r2_score, accuracy_score

In [ ]:
## Empiezo a importar las librerias necesarias para el proyecto
import matplotlib.pyplot as plt # Para graficar
import pandas as pd  # Para manipulacion de datos
import numpy as np   # Para operaciones numericas
import seaborn as sns
import lazypredict

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import RandomUnderSampler



In [ ]:
# Cargando Datasets...
train_df = pd.read_csv("train_users_2.csv")
test_df = pd.read_csv("test_users.csv")

# Mostrar informacion basica. Si deseo Mergearlos, necesito que sus vectores tengan la misma forma (1x16 , 1x15)
print("Train Shape:", train_df.shape)
print("Test Shape:", test_df.shape)


In [ ]:
#Muestro el nombre de todas las columnas disponibles en los dataframes, 
print("Train columns:\n", train_df.columns)
print("\nTest columns:\n", test_df.columns)

In [ ]:
print("\nTrain Data Types:\n", train_df.dtypes)
print("\nTest Data Types:\n", test_df.dtypes)

### Descripcion de 'train_set'
Es un vector Train Shape: (213451, 16)
Tengo 16 columnas:

    1. id                        9. language
    2. date_account_created      10. affiliate_channel  
    3. timestamp_first_active    11. affiliate_provider
    4. date_first_booking        12. first_affiliate_tracked
    5. gender                    13. signup_app
    6. age                       14. first_device_type
    7. signup_method             15. first_browser
    8. signup_flow               16. country_destination

Existen datos raros a simple vista?

### Descripcion de 'test_set'
Cuantas filas x columnas? Test Shape: (62096, 15)
Tengo 16 columnas:

    1. id                        9. language
    2. date_account_created      10. affiliate_channel  
    3. timestamp_first_active    11. affiliate_provider
    4. date_first_booking        12. first_affiliate_tracked
    5. gender                    13. signup_app
    6. age                       14. first_device_type
    7. signup_method             15. first_browser
    8. signup_flow               
Existen datos raros a simple vista?


Desde el vamos, el test_set carece de la columna Target, que seria 'country_destination'. Es decir: 

**No puedo hacer merge a estos datasets para obtener mas info para el entrenamiento de mi modelo.**

### Identificacion de Ruido en el Dataset

Empiezo buscando datos faltantes, nulos, corruptos, etc en ambos datasets:

In [ ]:
print("\nMissing Values in Train Set:\n", train_df.isnull().sum())
print("\nMissing Values in Test Set:\n", test_df.isnull().sum())


In [ ]:
a=213451 #Numero de filas en el dataset de entrenamiento
b= 100
c=train_df['date_first_booking'].isnull().sum()
x = (b*c)/a 
print("Perdida de  un ", x ," % de los datos")
print(c)

In [ ]:
d= 213451 #Numero de filas en el dataset de entrenamiento
e= 100
f= (train_df['country_destination']== 'NDF').sum()
y = (f*e)/d 
print("Perdida de  un ", y ," % de los datos")
print(f)

Estoy notando que en el archivo 'train_users_2' muchos usuarios NO tienen fecha de primera reserva. 

Como dato curioso, el dataset de entrenamiento tiene un total de 213451 filas, de las cuales:

    1. No existe ninguna fila en 'country_destination' que tenga valor Nulo y/o NaN
    2. Existen 124543 filas en 'first_date_booking' con valores Nulos y/o NaN
    3. Existen 124543 filas en 'country_destination' con valores 'NDF' o sin pais definido.

Si echo un vistazo en Excel y ordeno el dataset de menor a mayor fecha, todos los NDF coinciden con los registros que corresponden a 'first_date_booking' vacios

Como el objetivo de mis predicciones es conocer el destino de un usuario que viaja, podria droppear las filas de usuarios que NO tienen una reserva y observar si los resultados son buenos al momento de observar la distribucion. ---> Si este fuera el caso, **perderia 58.3473% de los datos**

En ese caso, decido dejar estos datos y compensarlo posteriormente con algun metodo ya sea undersampling, oversampling, class weights o afines.

Tambien observo que la columna 'age' tiene datos faltantes, sin embargo estos los voy a conservar, pues los puedo llenar con una mediana para no perder datos.



In [ ]:
categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()
categorical_summary = {col: train_df[col].nunique() for col in categorical_columns}

print("\nUnique Values in Categorical Features:\n", categorical_summary)

In [ ]:
print("\nAge Summary:\n", train_df['age'].describe())


In [ ]:
train_df['timestamp_first_active'] = pd.to_datetime(train_df['timestamp_first_active'], format='%Y%m%d%H%M%S')
test_df['timestamp_first_active'] = pd.to_datetime(test_df['timestamp_first_active'], format='%Y%m%d%H%M%S')

print("\nConverted Timestamp Example:\n", train_df[['timestamp_first_active']].head())


In [ ]:
# Summary of categorical features
categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()
categorical_summary = {col: train_df[col].nunique() for col in categorical_columns}

# Unique values in target variable
target_distribution = train_df['country_destination'].value_counts(normalize=True) * 100

# Checking age distribution and potential outliers
age_summary = train_df['age'].describe()

# Convert timestamp_first_active to datetime format
train_df['timestamp_first_active'] = pd.to_datetime(train_df['timestamp_first_active'], format='%Y%m%d%H%M%S')
test_df['timestamp_first_active'] = pd.to_datetime(test_df['timestamp_first_active'], format='%Y%m%d%H%M%S')

# Display results
categorical_summary, target_distribution, age_summary


Empezamos con el Analisis de Metadatos de mi dataset

Initial Findings:

    Dataset Sizes
        Train set: 213,451 rows × 16 columns
        Test set: 62,096 rows × 15 columns (missing country_destination, which is expected)

    Column Overview
        The datasets share the same structure except for country_destination in the train set.
        Most columns are categorical, with age as a numerical feature.
        timestamp_first_active is stored as an integer (likely a timestamp).

    Missing Values
        date_first_booking has 124,543 missing values (58%) in the train set and completely missing (100%) in the test set.
        age has 87,990 missing values (41%) in the train set and 28,876 missing (46%) in the test set.
        first_affiliate_tracked has some missing values (6,065 in train, 20 in test).

Next Steps:

    Check value distributions and unique categories.
    Investigate age (e.g., invalid values or outliers).
    Convert timestamp_first_active to a readable date.
    Examine the target variable country_destination.

I'll continue the analysis now. ​

​
Key Insights:

    Categorical Feature Overview
        gender: 4 unique values
        signup_method: 3 unique values
        language: 25 unique values
        first_browser: 52 unique values (could be simplified)
        country_destination: 12 unique values (our target variable)

    Target Variable Distribution
        58.3% of users did not book any destination (NDF)
        29.2% booked in the US
        Other countries have much lower representation (e.g., France: 2.35%, Italy: 1.33%)
        The dataset is highly imbalanced, which may require handling techniques (e.g., resampling or weighted loss functions).

    Age Distribution and Outliers
        Mean age: 49.67, but the max value is 2014, which is clearly incorrect.
        75% of users are 43 or younger, suggesting values above ~100 might be invalid.
        Likely need to filter or impute extreme age values.

    Timestamp Conversion
        timestamp_first_active is now a readable datetime format, which will allow extracting useful features (e.g., year, month, weekday).

## Pt. 2 EDA

Se realiza el tratamiento de los datos con el fin de mejorar la prediccion. 

### Visualizacion de Columna 'age'

Comienzo graficando los valores de la columna 'age' para conocer la distribucion de los datos:

In [ ]:
age = train_df['age']
print(train_df['age'].describe())

age.value_counts().plot.bar()

He decidido quitar los valores ilogicos de la columa age (por ejemplo, una persona de 2024 a;os de edad). Considero que para este analisis vale la pena pensar que las personas registradas deben tener al menos 18 a;os por motivos de politicas de la plataforma, y maximo 100 a;os como limite superior.

Hago asi para los conjuntos de test y train:

In [ ]:
# Convert timestamp_first_active to datetime format
train_df['timestamp_first_active'] = pd.to_datetime(train_df['timestamp_first_active'], format='%Y%m%d%H%M%S')

# Clean 'age' column: Keep values between 18 and 100, set others to NaN
valid_age_range_test = (train_df['age'] >= 18) & (train_df['age'] <= 100)
train_df.loc[~valid_age_range_test, 'age'] = np.nan

age.value_counts().plot.bar()
fig, ax = plt.subplots()
ax.hist(age.dropna(), bins=15, color='blue', alpha=0.7)

In [ ]:
aget = test_df['age']
aget.value_counts().plot.bar()

In [ ]:
# Convert timestamp_first_active to datetime format
test_df['timestamp_first_active'] = pd.to_datetime(test_df['timestamp_first_active'], format='%Y%m%d%H%M%S')

# Clean 'age' column: Keep values between 18 and 100, set others to NaN
valid_age_range_test = (test_df['age'] >= 18) & (test_df['age'] <= 100)
test_df.loc[~valid_age_range_test, 'age'] = np.nan

aget.value_counts().plot.bar()
fig, ax = plt.subplots()
ax.hist(aget, bins=15, color='blue', alpha=0.7)

### Visualizacion de Columna 'gender'

Grafico la columna de Genero del dataset, pues considero que esta variable puede ayudar a realizar predicciones.

In [ ]:
#Se grafican los valores unicos en la columna de genero, pues podria ser un buen indicador:
train_df['gender'].value_counts().plot.bar()
gender = train_df['gender']
fig, ax = plt.subplots()
ax.hist(gender, bins=15, color='blue', alpha=0.7)

In [ ]:
genderc = train_df['gender'].replace("-unknown-", np.nan)
gender.value_counts().plot.bar()
fig, ax = plt.subplots()
ax.hist(genderc.dropna(), bins=15, color='blue', alpha=0.7)

### Visualizacion de Columna 'signup_flow'

Esta columna podria ser util para el modelo predictivo, decido graficarla para observar la distribucion.

In [ ]:
sign = train_df['signup_flow']
sign.value_counts().plot.bar()
fig, ax = plt.subplots()
ax.hist(sign, bins=15, color='blue', alpha=0.7)

### Columna 'language'

Podria ser que el lenguaje del usuario de indicios sobre los paises que quiera visitar, posiblemente por razones culturales del lugar de origen. Pongamoslo a prueba.

Esta variable es multiple categorica, por lo que un histograma con su distribucion no seria util para saber que tan bien va a predecir.

In [ ]:
lang = train_df['language']
lang.value_counts().plot.bar()

Sin embargo, la inmensa mayoria de los usuarios es angloparlante, por lo cual no creo encontrar mucha informacion con estos datos.

### Columna 'country_destination'

Es mi columna objetivo para las predicciones, por lo cual debera ser manejada con mucho cuidado. Empezare por observar su histograma para determinar la distribucion de estos datos:

    Check for missing values (though there shouldn't be any in the training set).
    Look for inconsistencies (e.g., unexpected values).
    Analyze class distribution again (since it's highly imbalanced).
    Decide on encoding:
        We’ll likely use label encoding (mapping each country to a number) for modeling.
        Alternatively, one-hot encoding could be used if the model requires categorical features.

In [ ]:
destination = train_df['country_destination']
destination.value_counts().plot.bar()

La mayoria de los datos son a un pais NDF. Como he explorado antes, no puedo recortar los datos NDF, pues corresponden al 58.3~% de los datos del dataset. La mayoria de los usuarios cayendo en las categorias NDF y US. Esto tendra que ser compensado mas adelante en el proyecto con alguna medida. 

In [ ]:
# Check for missing values
print("Missing values in 'country_destination':", train_df['country_destination'].isnull().sum())

# Check unique values
print("\nUnique values in 'country_destination':", train_df['country_destination'].unique())

# Check target variable distribution
target_distribution = train_df['country_destination'].value_counts(normalize=True) * 100
print("\nTarget variable distribution (%):\n", target_distribution)

#target_distribution.plot()
plt.hist(target_distribution,15) ## Antes de graficar tengo que hacer hot encoding
plt.show

Estoy intentando observar la distribucion de una variable Categorica, lo cual no llegara muy lejos pues la distribucion aplica para valores numericos y continuos, no para categoricos.

## Pt. 3  Data Wrangling

En este apartado voy a procurar obtener datos con una distribucion mas cercana a lo que se considera Normal y dejarlos lo mas limpios posible para proponer un modelo predictivo.

### Columna 'age'

Procedo a tratar los datos del dataset de Entrenamiento imputando la mediana de edad en los valores vacios.

In [ ]:
# Fill missing ages with the same median from the training set (34)
median_age = 34  # Use the calculated median from train data
train_df['agef'] = train_df['age']

#Aqui sobreescribo la columna con datos imputados sobre la original
train_df['agef'].fillna(median_age, inplace=True)

# Verify the cleaned age distribution
print(train_df['agef'].describe())
valid_age_range_test = (train_df['agef'] >= 18) & (train_df['agef'] <= 100)
train_df.loc[~valid_age_range_test, 'agef'] = np.nan
train_df['agef'].value_counts().plot.bar()

fig, ax = plt.subplots()
ax.hist(train_df['agef'].dropna(), bins=15, color='blue', alpha=0.7)


In [ ]:
train_df.head()

In [ ]:
train_df['age'].head(20)

In [ ]:
train_df['agef'].head(20)

Despues de intentar reemplazar los valores NaN con la mediana de Edad, simplemente obtenemos un pico muy pronunciado con la distribucion aun cargada a la izquierda.

Intentare reiniciar mis valores y realizar un Sampleo Aleatorio (random sampling).

In [ ]:
# Extract valid (non-null) ages
valid_ages = train_df['age'].dropna()

# Apply random sampling for missing values
train_df['age_r']=train_df['age'].apply(lambda x: np.random.choice(valid_ages) if np.isnan(x) else x)
train_df['age_r'].describe()

fig, ax = plt.subplots()
ax.hist(train_df['age_r'], bins=15, color='blue', alpha=0.7)

balanced_df=train_df

balanced_df['age']=train_df['age_r']

In [ ]:
train_df['age_r'].head(30)

### Columna 'gender'

Considerando que respeto los valores 'male', 'female' y 'other' como variables categoricas, tengo que balancearlas de otra manera.



In [ ]:
from imblearn.under_sampling import RandomUnderSampler

X = train_df.drop(columns=['gender'])
y = train_df['gender']

undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)

balanced_df = X_resampled.copy()

## Guardo y sobreescribo la columna balanceada sobre la original

balanced_df['gender'] = y_resampled

y_resampled.head()
y_resampled.value_counts().plot.bar()

In [ ]:
balanced_df['gender'].head(20)

### Columna 'language'

Decido emplear RandomUnderSampling para balancear el sesgo pesado que los datos tienen hacia el idioma Ingles o 'en', en la columna:

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

X = train_df.drop(columns=['language'])
y = train_df['language']

undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)

balanced_df = X_resampled.copy()

## Guardo y sobreescribo la columna balanceada sobre la original

balanced_df['language'] = y_resampled

y_resampled.head()
y_resampled.value_counts().plot.bar()

In [ ]:
balanced_df['language'].tail(20)

### Columna 'Country Destination'

Empiezo a hacer el balanceo de la columna objetivo, empleando RandomUndersampling y dejar 

In [ ]:
# Separate features and target variable
X = train_df.drop(columns=['country_destination'])
y = train_df['country_destination']

# Apply undersampling
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)

# Recreate a balanced dataframe
balanced_df = pd.DataFrame(X_resampled, columns=X.columns)  # Reconstruct X as a DataFrame
balanced_df['country_destination'] = y_resampled  # Add the target column back

# Check the class distribution
y_resampled.value_counts().plot.bar()


In [ ]:
balanced_df['country_destination'].tail(40)

#### Dataframe Balanceado

Al final de cada columna procesada he estado guardando los datos "limpios" en las columnas de un dataframe nuevo llamado "balanced_df", que es el que voy a usar para mi procesamiento del modelo predictivo.
Empiezo quitando las columnas que no voy a usar para el analisis:

In [ ]:
balanced_df.drop(columns=['id','age_r' ], inplace=True)
balanced_df.head()

### Matriz de Correlaciones

Se crea una matriz de correlaciones para determinar cuales son las Features que podemos usar para predecir y cuales podemos dejar de un lado para optimizar el modelo:

In [ ]:
# Convert target variable to numerical encoding
balanced_df['country_destination'] = balanced_df['country_destination'].astype('category').cat.codes

# Select numerical features
numerical_features = balanced_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Compute correlation with target
correlation = balanced_df[numerical_features + ['country_destination']].corr()['country_destination'].sort_values(ascending=False)

# Display results
print(correlation)

In [ ]:
# Load dataset
# Convert target variable to numerical encoding
train_df['country_destination'] = train_df['country_destination'].astype('category').cat.codes

# Select numerical features
numerical_features = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Compute correlation with target
correlation = train_df[numerical_features + ['country_destination']].corr()['country_destination'].sort_values(ascending=False)

# Display results
print(correlation)

Comparado con el dataset original, podemos observar que la correlacion en las columnas del data set balanceado ha mejorado, aunque sea en poca medida.

## Pt. 4 - Diseño del Modelo Predictivo

En esta parte procedo a integrar modelos predictivos dise;ados para mi problema.

Tengo un problema: 
                    De aprendizaje supervisado: Conozco mi variable objetivo
                    Clasificatorio: Tengo que determinar en que categoria va a caer un usuario que viaja en AirBnB.
                    Multiclase: La variable objetivo se divide en 12 categorias distintas: AU, CA, DE, ES, FR, GB, IT, NL, PT, US, other, NDF.
                
En terminos de datos, existe la siguiente dificultad:

         El dataset 'test_users' NO posee una columna 'country_destination'.

Por lo cual tengo que apartar esos datos y particionar los datos en el conjunto de 'train_users_2' para poder entrenar un Modelo Predictivo y Validar dichas predicciones.


#### Particionando el Dataset 'train_users_2'

Decido hacer un particionado estratificado con un estado aleatorio predeterminado, para eso utilizare metodos en la libreria 'sklearn'

In [ ]:
#Quiero hacer un modelo predictivo usando mis datos ya procesados, son los siguientes:

balanced_df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Separate features and target
X = balanced_df.drop(columns=['country_destination'])  # Replace 'target_column_name' with the actual name
y = balanced_df['country_destination']

# Split the train set into training and validation subsets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


print(X)

In [ ]:
# Procesamiento y Encoding de Datos Categoricos
# Convert date columns to datetime

date_cols = ['date_account_created', 'timestamp_first_active', 'date_first_booking']
for col in date_cols:
    X_train[col] = pd.to_datetime(X_train[col], errors='coerce')
    X_val[col] = pd.to_datetime(X_val[col], errors='coerce')
    

In [ ]:
# Create numerical features from dates
for df in [X_train, X_val]:
    df['dac_year'] = df['date_account_created'].dt.year
    df['dac_month'] = df['date_account_created'].dt.month
    df['dac_day'] = df['date_account_created'].dt.day
    
    df['tfa_year'] = df['timestamp_first_active'].dt.year
    df['tfa_month'] = df['timestamp_first_active'].dt.month
    df['tfa_day'] = df['timestamp_first_active'].dt.day
    
    df['dfb_year'] = df['date_first_booking'].dt.year.fillna(0)  # Fill NaN with 0
    df['dfb_month'] = df['date_first_booking'].dt.month.fillna(0)
    df['dfb_day'] = df['date_first_booking'].dt.day.fillna(0)

# Drop original date columns
X_train.drop(columns=date_cols, inplace=True)
X_val.drop(columns=date_cols, inplace=True)



In [ ]:
X_train.head()

In [ ]:
X_val.head()

In [ ]:
# Identify categorical columns
categorical_cols = balanced_df.select_dtypes(include=['object']).columns.tolist()
print("Categorical Columns:", categorical_cols)


In [ ]:
from sklearn.preprocessing import LabelEncoder
## Se usa LabelEncoding para variables categoricas con multiples etiquetas
encoder = LabelEncoder()
for col in categorical_cols:
    balanced_df[col] = encoder.fit_transform(balanced_df[col])


In [ ]:
from sklearn.preprocessing import StandardScaler

# Identify numerical columns
numerical_cols = balanced_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Initialize scaler
scaler = StandardScaler()

# Scale numerical features
balanced_df[numerical_cols] = scaler.fit_transform(balanced_df[numerical_cols])


In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and target variable (y)
X = balanced_df.drop(columns=['country_destination'])
y = balanced_df['country_destination']

# Split data into 80% training and 20% validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Training set size:", X_train.shape)
print("Validation set size:", X_val.shape)


In [ ]:
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import accuracy_score


In [ ]:
# Initialize LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# Fit models
models, predictions = clf.fit(X_train, X_val, y_train, y_val)

# Display model performance
print(models)


Podemos ver que el top 3 de algoritmos (ordenados de mayor eficiencia a menor), son:

                                    Accuracy    Balanced Accuracy  ROC AUC   F1 Score
    1. BaggingClassifier                  0.20               0.20    None      0.20   
    2. DecisionTreeClassifier             0.20               0.20    None      0.20   
    3. RandomForestClassifier             0.18               0.18    None      0.18     

Sin embargo, estos porcentajes de precision son horribles. Procedo a intentar otro algoritmo con otro tipo de balanceo.

### Intentando XGBoost

Adapto el dataset balanceado a un formato que puede leer XGBoost:


#### Pruebas de XGBoost con el Dataset recortado

Voy a probar este algoritmo con el Dataset Balanceado que he generado, cabe mencionar que recorte algunas columnas para intentar mejorar el tiempo de procesamiento y eliminar el ruido:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
data = balanced_df
# Preprocessing
# Drop rows with missing target values
data = data.dropna(subset=['country_destination'])

# Handle missing values in features
data = data.fillna(-1)

# Encode categorical variables
categorical_features = ['gender', 'language']
for feature in categorical_features:
    le = LabelEncoder()
    data[feature] = le.fit_transform(data[feature].astype(str))

# Encode the target variable
le = LabelEncoder()
data['country_destination'] = le.fit_transform(data['country_destination'])

# Split the data into features and target
X = data.drop(['date_account_created', 'timestamp_first_active', 'date_first_booking', 'country_destination'], axis=1)
y = data['country_destination']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
model = XGBClassifier(objective='multi:softprob', num_class=len(le.classes_), random_state=42)
model.fit(X_train, y_train)

# Validate the model
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.4f}')

# If you want to see the predicted classes in their original form
y_pred_original = le.inverse_transform(y_pred)
print(y_pred_original[:10])  # Print the first 10 predictions

### Resultados de XGBoost con pocos datos

El resultado de la validacion para este conjunto de datos 
        'balanced_df'
Fue de un vergonzoso 10.36%. Por lo cual podemos volver a concluir:

                        "En Data Science, cuanto mas tengas, mejor."


#### Prueba XGBoost con Dataset con mas columnas

Ahora intento el mismo algoritmo, pero sin amputar tanta informacion, pues los resultados previos fueron vergonzosos.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
data = pd.read_csv('train_users_2.csv')
# Preprocessing
# Drop rows with missing target values
data = data.dropna(subset=['country_destination'])

# Handle missing values in features
data = data.fillna(-1)

# Encode categorical variables
categorical_features = ['gender', 'signup_method', 'signup_app', 'first_device_type', 'first_browser', 
                        'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'language']
for feature in categorical_features:
    le = LabelEncoder()
    data[feature] = le.fit_transform(data[feature].astype(str))

# Encode the target variable
le = LabelEncoder()
data['country_destination'] = le.fit_transform(data['country_destination'])


# Split the data into features and target
X = data.drop(['id', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 'country_destination'], axis=1)
y = data['country_destination']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
model = XGBClassifier(objective='multi:softprob', num_class=len(le.classes_), random_state=42)
model.fit(X_train, y_train)

# Validate the model
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.4f}')

# If you want to see the predicted classes in their original form
y_pred_original = le.inverse_transform(y_pred)
print(y_pred_original[:10])  # Print the first 10 predictions

In [ ]:
##Voy a respetar la mayor parte de las columnas posible en vez de droppear las que empiricamente se quitarian

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score


# Load the dataset
data = pd.read_csv('train_users_2.csv')

# Preprocessing
# Drop rows with missing target values
data = data.dropna(subset=['country_destination'])

### Voy a droppear valores NaN de la columna date_fist_booking
# data = data.dropna(subset=['date_first_booking'])
### Resulta en un bajon de Accuracy, baja a un 69.9%

# Handle missing values in features
data = data.fillna(-1)

# Feature Engineering for date columns
# Convert to datetime
data['date_account_created'] = pd.to_datetime(data['date_account_created'])
data['timestamp_first_active'] = pd.to_datetime(data['timestamp_first_active'], format='%Y%m%d%H%M%S')
data['date_first_booking'] = pd.to_datetime(data['date_first_booking'])

# Extract features from date_account_created
data['account_created_year'] = data['date_account_created'].dt.year
data['account_created_month'] = data['date_account_created'].dt.month
data['account_created_day'] = data['date_account_created'].dt.day
data['account_created_weekday'] = data['date_account_created'].dt.weekday  # Monday=0, Sunday=6

# Extract features from timestamp_first_active
data['first_active_year'] = data['timestamp_first_active'].dt.year
data['first_active_month'] = data['timestamp_first_active'].dt.month
data['first_active_day'] = data['timestamp_first_active'].dt.day
data['first_active_hour'] = data['timestamp_first_active'].dt.hour

# Extract features from date_first_booking (if available)
data['booking_year'] = data['date_first_booking'].dt.year
data['booking_month'] = data['date_first_booking'].dt.month
data['booking_day'] = data['date_first_booking'].dt.day
data['booking_weekday'] = data['date_first_booking'].dt.weekday

# Calculate time differences
data['time_to_first_booking'] = (data['date_first_booking'] - data['date_account_created']).dt.days
data['time_since_account_created'] = (pd.Timestamp.now() - data['date_account_created']).dt.days

# Drop the original date columns 
data = data.drop(['date_account_created', 'timestamp_first_active', 'date_first_booking'], axis=1)

# Encode categorical variables
## Procuro mantener la mayor cantidad de columnas posibles
categorical_features = ['gender', 'signup_method', 'signup_app', 'first_device_type', 'first_browser', 
                        'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'language']
for feature in categorical_features:
    le = LabelEncoder()
    data[feature] = le.fit_transform(data[feature].astype(str))

# Encode the target variable
le = LabelEncoder()
data['country_destination'] = le.fit_transform(data['country_destination'])

# Split the data into features and target
X = data.drop(['id', 'country_destination'], axis=1)  # Drop 'id' and the target column
y = data['country_destination']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
model = XGBClassifier(objective='multi:softprob', num_class=len(le.classes_), random_state=42)
model.fit(X_train, y_train)

# Validate the model
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.4f}')

# If you want to see the predicted classes in their original form
y_pred_original = le.inverse_transform(y_pred)
print(y_pred_original[:10])  # Print the first 10 predictions

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 6],
    'subsample': [0.8, 1.0]
}

# Perform GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters and accuracy
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)


In [ ]:
# Evaluate on test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_train)
test_accuracy = accuracy_score(y_train, y_pred)
print("Test Set Accuracy:", test_accuracy)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
# Make predictions
y_pred = best_model.predict(X_train)

# Compute evaluation metrics
accuracy = accuracy_score(y_train, y_pred)
conf_matrix = confusion_matrix(y_train, y_pred)
precision = precision_score(y_train, y_pred, average='weighted')  # Use 'macro' for class imbalance
recall = recall_score(y_train, y_pred, average='weighted')
f1 = f1_score(y_train, y_pred, average='weighted')

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Plot Confusion Matrix
plt.figure(figsize=(10, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_train), yticklabels=np.unique(y_train))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

### Importancia de Features

Procedo a graficar las variables de fechas y determinar cuanto impactan en el conjunto de datos:

In [ ]:
import matplotlib.pyplot as plt
from xgboost import plot_importance

plot_importance(model)
plt.show()

Se puede apreciar en estos datos, aquellos que tienen mas peso son los de 'Edad'.

### Resultados XGBoost
Usando XGBoost con la mayor cantidad de columnas posibles y sin amputar datos, obtengo una 

                                            Accuracy de 87.5%


Empleando GridSearch para ajustar hiperparametros, el tiempo de procesamiento es muchisimo mas alto. 
Y el modelo mejora hasta un... 

                                            Accuracy de 87.579%

Lo cual representa una mejora, sin embargo no considero que el tiempo de procesamiento amerite el incremento en Accuracy.

Finalmente, corroborando el modelo que acabo de realizar, se obtienen las siguientes metricas.

✓ Accuracy: 0.8758
✓ Precision: 0.7887
✓ Recall: 0.8758
✓ F1 Score: 0.8246

Los cuales aseguran que el modelo predictivo es hasta cierto punto confiable:
El modelo aprendió patrones fuertes en los datos y tiene una alta capacidad de predicción asi un buen balance entre precisión y recall. Sin embargo, al tener una precisión más baja que recall, puede estar prediciendo destinos erróneos en algunos casos.

## Conclusiones

1. Comparando el performance de XGBoost sin eliminar tantas filas (train_users_2.csv)
vs el dataset recortado (balanced_df), el dataset con mas informacion tiene muchisima mejor precision que el balanceado. Mientras mas informacion respete, mejor.
2. Lazypredict ayuda a probar multiples metodos de machine learning y evaluar su eficiencia de una manera muy conveniente, tomando en cuenta que el usuario no tiene que escribir tantas lineas de codigo.
3. El balanceo de informacion es importante, pero si quitas muchas columnas, terminaras con un modelo PESIMO.
4. De nuevo, en el mundo de ciencia de datos: a mayor cantidad, mejor. Claro, que la limpieza de los datos es importante, pero si tienes muchisimos datos, tienes mas posibilidades de hallar algo bueno.
5. La afinacion de hiperparametros puede ser muy util para mejorar el rendimiento del modelo, pero a veces es tan pequenya la mejoria, que no vale la pena aumentar el tiempo de procesamiento.


